# **USECASE: Building a disease-disease similairty netwrok**

* DBRetina is an efficent tool for  building a similarity network for a set of items by pairwaise calcuation of their shared features using a linear-time algorithm.
* [DisGeNET](https://www.disgenet.org/) has one of the largest collections of genes associated to human diseases.
* In this tutorial, we will use DBRetina to build a disease-disease similairty netwrok based on the shared number of genes asscoiated with them in the DisGeNET database

In [ ]:
%%bash
## Download the most recent version of disease gene associations from DisGeNET
if [ ! -f all_gene_disease_associations.tsv ];then
  wget -N http://www.disgenet.org/static/disgenet_ap1/files/downloads/all_gene_disease_associations.tsv.gz
  gunzip all_gene_disease_associations.tsv.gz
else echo "all_gene_disease_associations.tsv file exists in the disgenet DB";fi

In [ ]:
%%bash
head -n3 all_gene_disease_associations.tsv

In [ ]:
%%bash
## transform the data table into the DBRetina format
## DBRetina expects 2 files. Both are tab-separated files with two columns. Files must have header lines
## 1) Associations file: The 1st column for "items" and the 2nd for their asscoiated "features".
## 2) Super-association file: The 1st column for "items" and the 2nd for their "aliases". You can use this column to update the item name or if you want to pool multiple items together as one super item otherwise the 2nd column should be a copy of the 1st column
## In addition, we will filter the input list to keep trusted disease-gene associations only (DisGeNET score > 0.3)
cat all_gene_disease_associations.tsv | sed -e 's/^[ \t]*//' | awk 'BEGIN{FS=OFS="\t";}{if($10>0.3)print $6,$2}' > disgenet.asc
echo "item alias" | tr ' ' '\t' > disgenet.names
tail -n+2 disgenet.ld | awk 'BEGIN{FS=OFS="\t";}{print $1,$1}' | sort | uniq >> disgenet.names

In [ ]:
%%bash
## Let us explore the format of the prepared filed
echo "DisGeNET input file"
wc -l all_gene_disease_associations.tsv
echo "==================="
echo "Associations file"
wc -l disgenet.asc
head -n3 disgenet.asc
echo "==================="
echo "Super-associations file"
wc -l disgenet.names
head -n3 disgenet.names

In [ ]:
%%bash
## Now we can run DBRetina
DBRetina items_indexing -i disgenet.asc -n disgenet.names -p disgenetDBR
DBRetina pairwise -i disgenetDBR
head disgenetDBR_kSpider_pairwise.tsv